<h1><center>EDA on Bank Marketing Dataset</center></h1>

<center>https://archive.ics.uci.edu/ml/datasets/Bank+Marketing</center>

## 1. Random Forests with untransformed data

In [1]:
# Jupyter Notebook with Matplotlib Inline
%matplotlib notebook

# Importing necessary modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

height has been deprecated.



In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from pprint import pprint

In [4]:
# Reading the csv file.
df_test = pd.read_csv("data/cleaned/test.csv")
df_train = pd.read_csv("data/cleaned/train.csv")

In [5]:
# Concatenated data
df_full = pd.concat([df_test, df_train], axis = 0)

In [6]:
# Verifying if the data has been contacted by verifying the number of rows
print((len(df_test) + len(df_train)) == len(df_full))

True


In [7]:
print(df_full.select_dtypes(include=['O']).columns.values)

['job' 'marital' 'education' 'default' 'housing' 'personal' 'contact_type'
 'month' 'day' 'poutcome' 'y']


In [8]:
categorical =['job', 'marital', 'education', 'default', 'housing', 'personal', 'contact_type',
 'month', 'day', 'poutcome']

In [9]:
all_dataframes = [df_full]

In [10]:
# Get dummies for all_dataframes_1 with categorical_fields_1
df_full = pd.get_dummies(df_full, columns = categorical)

In [11]:
all_dataframes = [pd.get_dummies(df, columns = categorical) for df in all_dataframes]

In [12]:
# Necessary imports
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from pprint import pprint

In [138]:
# Train and Test data
X = df_full.drop(['y'], axis = 1)
y = pd.get_dummies(df_full[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting dataframe to numpy array
data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)

# Instantiate model with 1000 decision trees
rf_df_full = RandomForestClassifier(n_estimators = 1000, random_state = 42)

# Train the model on training data
rf_df_full.fit(X_train, y_train.values.ravel());

# Predict test data
pred = rf_df_full.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.912357368293
Precision --  0.445974576271
Recall --  0.679032258065
F1 Score --  0.538363171355
AUC --  51838.5


In [14]:
print(metrics.classification_report(pred, y_test))

             precision    recall  f1-score   support

          0       0.97      0.93      0.95      7618
          1       0.45      0.68      0.54       620

avg / total       0.93      0.91      0.92      8238



## 1.1 RandomizedSearchCV

In [15]:
from sklearn.ensemble import RandomForestRegressor
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV

rf = RandomForestRegressor(random_state = 42)

# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_split': 1e-07,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [119]:
# Number of trees in random forest
n_estimators = [100, 200, 400, 600, 800, 1000]

# Number of features to consider at every split
max_features = ['auto', 'sqrt', 0.2]

# Maximum number of levels in tree
max_depth = [1, 2, 3, 4, 5, 10, 25, 50, 75, 100, 110, None]

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 5, 10, 50, 100, 200, 500]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [1, 2, 3, 4, 5, 10, 25, 50, 75, 100, 110, None],
 'max_features': ['auto', 'sqrt', 0.2],
 'min_samples_leaf': [1, 2, 4, 5, 10, 50, 100, 200, 500],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [100, 200, 400, 600, 800, 1000]}


In [120]:
# First create the base model to tune
rf = RandomForestRegressor()

# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)

# Fit the random search model
rf_random.fit(X_train, y_train.values.ravel())

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 14.7min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 22.6min finished


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False),
          fit_params={}, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [100, 200, 400, 600, 800, 1000], 'max_features': ['auto', 'sqrt', 0.2], 'max_depth': [1, 2, 3, 4, 5, 10, 25, 50, 75, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4, 5, 10, 50, 100, 200, 500], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score=True, scoring=None, verbose=2)

In [121]:
rf_random.best_params_

{'bootstrap': False,
 'max_depth': 110,
 'max_features': 0.2,
 'min_samples_leaf': 5,
 'min_samples_split': 5,
 'n_estimators': 800}

** Evaluate Random Search: **

In [122]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = ', accuracy_score(pred, y_test)*100)
    return (accuracy_score(pred, y_test)*100)

In [123]:
base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
base_model.fit(X_train, y_train.values.ravel())
base_accuracy = evaluate(base_model, X_test, y_test.values.ravel())

Model Performance
Average Error: 0.1779 degrees.
Accuracy =  88.0387931034


C:\Users\Nishu\Anaconda3\Lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
C:\Users\Nishu\Anaconda3\Lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [124]:
best_random = rf_random.best_estimator_
random_acc = evaluate(best_random, X_test, y_test.values.ravel())

Model Performance
Average Error: 0.1866 degrees.
Accuracy =  88.0387931034


C:\Users\Nishu\Anaconda3\Lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.


In [126]:
# Get dummies for all_dataframes_1 with categorical_fields_1
df_full = pd.get_dummies(df_full, columns = categorical)

** Random Search Best Params **

In [139]:
X = df_full.drop(['y'], axis = 1)
y = pd.get_dummies(df_full[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)

# Instantiate model with 1000 decision trees
df_full_best_model = RandomForestClassifier(bootstrap = False, max_depth = 110 ,
 max_features = 0.2,
 min_samples_leaf = 5,
 min_samples_split = 5,
 n_estimators = 800, random_state = 42)

# Train the model on training data
df_full_best_model.fit(X_train, y_train.values.ravel());

# Predict test data
pred = df_full_best_model.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.916484583637
Precision --  0.503177966102
Recall --  0.684438040346
F1 Score --  0.579975579976
AUC --  56956.5


** Random Search Base Params **

In [140]:
X = df_full.drop(['y'], axis = 1)
y = pd.get_dummies(df_full[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)

# Instantiate model with 1000 decision trees
df_full_base_model = RandomForestClassifier(n_estimators = 1000, random_state = 42)

# Train the model on training data
df_full_base_model.fit(X_train, y_train.values.ravel());

# Predict test data
pred = df_full_base_model.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.912357368293
Precision --  0.445974576271
Recall --  0.679032258065
F1 Score --  0.538363171355
AUC --  51838.5


In [24]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_acc - base_accuracy) / base_accuracy))

Improvement of 0.00%.


## 1.2 Grid Search with Cross Validation

In [131]:
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4],
    'min_samples_split': [8, 10],
    'n_estimators': [100, 200, 300]
}

# Create a based model
rf = RandomForestRegressor()

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)

In [132]:
X = df_full.drop(['y'], axis = 1)
y = pd.get_dummies(df_full[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)
    
# Fit the grid search to the data
grid_search.fit(X_train, y_train)

grid_search.best_params_

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 13.2min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed: 18.0min finished
C:\Users\Nishu\Anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  best_estimator.fit(X, y, **self.fit_params)


{'bootstrap': True,
 'max_depth': 110,
 'max_features': 3,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 100}

** Grid Search Best Params **

In [141]:
X = df_full.drop(['y'], axis = 1)
y = pd.get_dummies(df_full[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)

# Params of best model in Grid search
df_full_best_model = RandomForestClassifier(bootstrap = False, max_depth = 110 ,
 max_features = 3,
 min_samples_leaf = 5,
 min_samples_split = 8,
 n_estimators = 100, random_state = 42)

# Train the model on training data
df_full_best_model.fit(X_train, y_train.values.ravel());

# Predict test data
pred = df_full_best_model.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.906652100024
Precision --  0.273305084746
Recall --  0.756598240469
F1 Score --  0.401556420233
AUC --  32250.5


** Grid Search Base Model **

In [142]:
X = df_full.drop(['y'], axis = 1)
y = pd.get_dummies(df_full[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)

# Params of base model in Random search
df_full_best_model = RandomForestClassifier(random_state = 42)

# Train the model on training data
df_full_best_model.fit(X_train, y_train.values.ravel());

# Predict test data
pred = df_full_best_model.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.908351541636
Precision --  0.404661016949
Recall --  0.664347826087
F1 Score --  0.502962475313
AUC --  46974.0


In [27]:
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, X_test, y_test.values.ravel())

Model Performance
Average Error: 0.1337 degrees.
Accuracy =  91.2357368293


C:\Users\Nishu\Anaconda3\Lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.


In [28]:
print('Improvement of {:0.2f}%.'.format( 100 * (grid_accuracy - base_accuracy) / base_accuracy))

Improvement of 0.00%.


<div class="alert alert-success">
** Note: ** Since the goal of this project is to minimize False Negatives (How many did we miss), we focus on getting a recall value close to 100% with a less bad precision value
</div>

After builds models with various transformations, feature engineering techniques and their combinations, here are that were selected based on recall, precision and f1 scores.

<img src="img/models and scores.png">

** Note: ** <br>
Primary Models - Highlighted in yellow <br>
Experimenting Models - Highlighted in blue

** Hence, proceeding with models builts for the above dataframes. **

## 2. Transformations and Random Forests on the transformed datasets

### 2.1 Data Transformation

**Consolidating and Categorizing:**

    a. 'job', 'education' and 'month' -- Consolidating these variables on percentage of positive and negative responses.
    b. 'age' -- Binning and Categorizing the ages into a 'young_adult', 'adult' and 'senior' using 'qcut'. 
    c. 'day' -- Categorizing day into 'weekday_1', 'weekday_2' and 'weekend' categories.
    
**Merging two features:**

    a. 'age' and 'marital' -- Merging two variable to create a new.

**Dropping a column:** 

    a. 'duration' -- Create two dataframes (one with 'duration' column and another without). This is covered at the end of this section.

In [29]:
# Reading the csv file.
df_test = pd.read_csv("data/cleaned/test.csv")
df_train = pd.read_csv("data/cleaned/train.csv")

In [30]:
# Concatenated data
df_full = pd.concat([df_test, df_train], axis = 0)

** Transformation 1: Categorizing `job`, `education` and `month`**

In [31]:
df_transformations = df_full.copy(deep = True)

In [32]:
# Function to categorize 'job', 'education' and 'month'
def transformation_1(df):

    col = ['job', 'education', 'month']

    for c in col:
        if c == 'job':
            df['job'].replace(['blue-collar', 'services', 'entrepreneur', 'housemaid', 'self-employed', 'technician', 
                  'management', 'unknown', 'admin.', 'unemployed', 'retired', 'student'],
                   ['jl4', 'jl4', 'jl4', 'jl3', 'jl3', 'jl3', 'jl3', 'jl2', 'jl2', 'jl2', 'jl1', 'jl1'], 
                   inplace=True)

        elif c == 'education':
            df['education'].replace(['basic.9y','basic.6y','basic.4y','high.school','professional.course','university.degree','unknown','illiterate'],
                      ['el4','el4','el3','el3','el3','el2','el2','el1'], 
                      inplace=True)

        elif c == 'month':
            df['month'].replace(['may','jul','nov','aug','jun','apr','oct','sep','dec','mar'],['ml3','ml3','ml3','ml3','ml3','ml2','ml1','ml1','ml1','ml1'], 
                      inplace=True)

** Transformation 2: Categorizing `age`**

In [33]:
# Function to categorize 'age'
def transformation_2(df):
    age_groups = ['young_adult', 'adult', 'senior']
    df['age_group'] = pd.qcut(df['age'], 3, labels = age_groups)
    df['age_group'] = pd.Categorical(df.age_group)

** Transformation 3 - Categorizing 'day' variable **

In [34]:
# Function to categorize 'day'
def transformation_3(df):
    df['day'].replace(['sum', 'sat', 'mon', 'tue', 'wed', 'thu', 'fri'],
                      ['weekend', 'weekend', 'weekday_1', 'weekday_1', 'weekday_1', 'weekday_2', 'weekday_2'], 
                      inplace = True)

** Transformation 4: Merging `marital` and `age`**

In [36]:
pd.crosstab(df_transformations.marital, df_transformations.age)

age,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,91,92,94,95,98
marital,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
divorced,0,0,0,0,0,0,0,4,17,13,14,21,67,93,141,129,140,141,166,153,158,171,152,154,193,173,114,155,184,157,157,144,150,121,132,140,142,132,124,131,118,100,87,44,5,4,4,4,3,7,5,6,6,11,3,7,8,9,8,4,7,10,0,14,9,6,7,5,3,2,1,18,2,0,1,0,1,0
married,0,0,0,1,8,16,30,78,150,196,252,372,529,758,909,936,1035,1071,1076,1136,914,872,979,777,864,797,783,686,792,762,671,743,615,670,580,584,544,513,487,543,468,448,357,229,65,57,51,53,39,43,20,23,28,36,44,25,25,23,16,29,12,17,14,17,11,9,10,2,11,5,0,4,0,2,3,1,0,2
single,5,28,42,64,94,121,196,381,429,489,583,608,852,859,891,776,652,526,514,489,399,362,301,229,220,172,158,170,124,110,100,92,70,79,42,52,46,37,35,29,58,25,18,10,3,1,0,0,2,5,1,4,0,0,6,2,1,0,0,1,1,0,0,0,0,2,0,0,1,1,0,0,0,0,0,0,0,0
unknown,0,0,0,0,0,0,0,0,2,0,2,0,5,4,6,5,6,7,3,2,4,2,0,1,1,0,0,0,3,1,0,0,4,5,0,3,1,2,2,1,2,3,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [35]:
# Function to categorize 'marital' and 'age'
def transformation_4(df):
    df['age_marital'] = df.apply(lambda x: x['age_group'] + ' & ' + x['marital'], axis = 1)

### 2.1.1 Applying transformations

The transformed data can be further divided into two dataframes:

1. df_age_1 - with `age` 
2. df_age_2 - with `age_group`
2. df_age_marital - with `age_marital`

In [36]:
# Dataframes to apply transformations
df_age_1 = df_full.copy(deep = True)
df_age_2 = df_full.copy(deep = True)
df_age_marital = df_full.copy(deep = True)

In [37]:
dataframes = [df_age_1, df_age_2, df_age_marital]

In [38]:
# Function to apply all transformations
def transformations(df):
    transformation_1(df)
    transformation_2(df)
    transformation_3(df)
    transformation_4(df)

In [39]:
# Applying transformations to 'df_age_1', 'df_age_2' and 'df_age_marital'
for df in dataframes:
    transformations(df)

In [48]:
df_age_marital[['age', 'age_marital', 'age_group', 'day', 'education', 'job', 'month', 'marital']].head()

,age,age_marital,age_group,day,education,job,month,marital
0,36,adult & married,adult,weekday_1,el4,jl4,ml3,married
1,60,senior & married,senior,weekday_1,el3,jl2,ml3,married
2,45,senior & married,senior,weekday_1,el3,jl4,ml3,married
3,27,young_adult & married,young_adult,weekday_2,el3,jl2,ml3,married
4,38,adult & married,adult,weekday_2,el2,jl2,ml3,married


** Dropping unwanted columns: **

In [49]:
print(df_transformations.select_dtypes(include=['O']).columns.values)

['job' 'marital' 'education' 'default' 'housing' 'personal' 'contact_type'
 'month' 'day' 'poutcome' 'y']


In [40]:
categorical_1 = ['job', 'marital', 'education', 'default', 'housing', 'personal', 'contact_type',
 'month', 'day', 'poutcome']

categorical_2 = ['job', 'marital', 'education', 'default', 'housing', 'personal', 'contact_type',
 'month', 'day', 'poutcome', 'age_group']

categorical_3 = ['job', 'education', 'default', 'housing', 'personal', 'contact_type',
 'month', 'day', 'poutcome', 'age_marital']

In [41]:
# Dropping respective columns 
df_age_1 = df_age_1.drop(['age_group', 'age_marital'], axis = 1)
df_age_2 = df_age_2.drop(['age', 'age_marital'], axis = 1)
df_age_marital = df_age_marital.drop(['age', 'age_group', 'marital'], axis = 1)

In [42]:
# Get dummies for the dataframes
df_age_1 = pd.get_dummies(df_age_1, columns = categorical_1)
df_age_2 = pd.get_dummies(df_age_2, columns = categorical_2)
df_age_marital = pd.get_dummies(df_age_marital, columns = categorical_3)

<div class="alert alert-info">
If the above piece of code does not create dummy variables, use the following code snippet: <br>
<br>
all_dataframes = [pd.get_dummies(df, columns = categorical) for df in all_dataframes]
</div>

### Random Forest on `df_age_1`

In [43]:
# Train and Test data
X = df_age_1.drop(['y'], axis = 1)
y = pd.get_dummies(df_age_1[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting dataframe to numpy array
data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)
    
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.values.ravel());

# Predict test data
pred = rf.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

0.915513474144


In [44]:
# Classification Report
print(metrics.classification_report(pred, y_test))

             precision    recall  f1-score   support

          0       0.97      0.94      0.95      7558
          1       0.49      0.68      0.57       680

avg / total       0.93      0.92      0.92      8238



** Random Search - Base Params **

In [143]:
# Train and Test data
X = df_age_1.drop(['y'], axis = 1)
y = pd.get_dummies(df_age_1[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting dataframe to numpy array
data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)
    
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.values.ravel());

# Predict test data
pred = rf.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.907259043457
Precision --  0.407838983051
Recall --  0.652542372881
F1 Score --  0.501955671447
AUC --  47360.0


** Random Search - Best Params **

In [144]:
# Train and Test data
X = df_age_1.drop(['y'], axis = 1)
y = pd.get_dummies(df_age_1[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting dataframe to numpy array
data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)
    
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(bootstrap = False, max_depth = 110,
 max_features = 0.2,
 min_samples_leaf = 5,
 min_samples_split = 5,
 n_estimators = 800, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.values.ravel());

# Predict test data
pred = rf.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.917212915756
Precision --  0.512711864407
Recall --  0.685552407932
F1 Score --  0.586666666667
AUC --  58362.5


** Grid Search - Best Params **

In [146]:
# Train and Test data
X = df_age_1.drop(['y'], axis = 1)
y = pd.get_dummies(df_age_1[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting dataframe to numpy array
data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)
    
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(bootstrap = False, max_depth = 110 ,
 max_features = 3,
 min_samples_leaf = 5,
 min_samples_split = 8,
 n_estimators = 100, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.values.ravel());

# Predict test data
pred = rf.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.909079873756
Precision --  0.323093220339
Recall --  0.734939759036
F1 Score --  0.448859455482
AUC --  37883.0


### Random Forest on `df_age_2`

In [145]:
# Train and Test data
X = df_age_2.drop(['y'], axis = 1)
y = pd.get_dummies(df_age_2[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting dataframe to numpy array
data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)

# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.values.ravel())

# Predict test data
pred = rf.predict(X_test)

# Accuray Score
print(accuracy_score(pred, y_test))

Accuracy --  0.914299587278
Precision --  0.491525423729
Recall --  0.672463768116
F1 Score --  0.567931456548
AUC --  56701.0


In [46]:
# Classification Report
print(metrics.classification_report(pred, y_test))

             precision    recall  f1-score   support

          0       0.97      0.94      0.95      7548
          1       0.49      0.67      0.57       690

avg / total       0.93      0.91      0.92      8238



** Random Search - Base Params **

In [149]:
# Train and Test data
X = df_age_2.drop(['y'], axis = 1)
y = pd.get_dummies(df_age_2[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting dataframe to numpy array
data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)
    
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.values.ravel());

# Predict test data
pred = rf.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.908108764263
Precision --  0.423728813559
Recall --  0.652528548124
F1 Score --  0.513808606294
AUC --  48131.5


** Random Search - Best Params **

In [150]:
# Train and Test data
X = df_age_2.drop(['y'], axis = 1)
y = pd.get_dummies(df_age_2[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting dataframe to numpy array
data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)
    
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(bootstrap = False, max_depth = 110,
 max_features = 0.2,
 min_samples_leaf = 5,
 min_samples_split = 5,
 n_estimators = 800, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.values.ravel());

# Predict test data
pred = rf.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.91709152707
Precision --  0.507415254237
Recall --  0.687230989957
F1 Score --  0.583790371725
AUC --  57850.0


** Grid Search - Best Params **

In [151]:
# Train and Test data
X = df_age_2.drop(['y'], axis = 1)
y = pd.get_dummies(df_age_2[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting dataframe to numpy array
data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)
    
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(bootstrap = False, max_depth = 110 ,
 max_features = 3,
 min_samples_leaf = 5,
 min_samples_split = 8,
 n_estimators = 100, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.values.ravel());

# Predict test data
pred = rf.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.908837096383
Precision --  0.301906779661
Recall --  0.755968169761
F1 Score --  0.431491294474
AUC --  35069.0


### 2.2 Outliers Treatment

In [152]:
import math

In [153]:
# Function to replace outliers with Upper and Lower bounds for 'duration' and 'employee' column
def outliers_bounds(df):
    df['duration'] = df['duration'].apply(lambda x: int(math.floor(x / 10.0)) * 10 if(x%10<5) else int(math.ceil(x / 10.0)) * 10 )
    df['employees'] = df['employees'].apply(lambda x: int(math.floor(x / 10.0)) * 10 if(x%10<5) else int(math.ceil(x / 10.0)) * 10 )

In [49]:
# Function to replace outliers with Replace outliers with upper and lower quartiles (`percentiles`)
def outliers_quartiles(df):
    colz = ['duration', 'dcontacts', 'pdays', 'evr', 'cpi', 'cci', 'euribor', 'employees']
    uq = 0.95
    lq = 0.05
    for col in colz:
        df[col] = df[col].clip_upper(int(df[col].quantile(uq)))
        df[col] = df[col].clip_lower(int(df[col].quantile(lq)))

In [50]:
# Function to replace outliers with Apply Logarithmic transformations (Not just to the outliers but to all the samples (observations) in the numeric columns). 
def outliers_log(df):
    num = ['age','dcontacts', 'cpi', 'euribor','employees']
    for n in num:
        df[n] = np.log(df[n])

### 2.2.1 Applying Outlier Treatment

The transformed data can be further divided into two dataframes:

1. df_outliers_1 - outluiers treatment 
2. df_outliers_2 - outliers and transformations
2. df_outliers_3 - outliers, transformations and log

In [54]:
# Function to perform outlier treatment
def outliers(df):
        outliers_bounds(df)
        outliers_quartiles(df)

## 3. Data Scaling

### 3.1 Normalization

In [55]:
# Numerical Columns
numerical = ['age', 'duration', 'dcontacts', 'pdays', 'pcontacts', 'evr', 'cpi', 'cci', 'euribor', 'employees']

In [56]:
# Function to perform normalization
def normalization(df):
    for n in numerical:
        col = df[[n]].values.astype(float)
        col_transformed = (preprocessing.MinMaxScaler()).fit_transform(col)
        # df[n+'_normalized'] = pd.df(col_transformed)
        df[n] = pd.DataFrame(col_transformed)

### 3.2 Standardization

In [58]:
# Function to perform standardization
def standardization(df):
    for n in numerical:
        col = df[[n]].values.astype(float)
        col_transformed = (preprocessing.StandardScaler()).fit_transform(col)
        # df[n+'_standardized'] = pd.DataFrame(col_transformed)
        df[n] = pd.DataFrame(col_transformed)

## 4. Resampling

### 4.1 Upsample

In [60]:
from sklearn.utils import resample
from sklearn import preprocessing

In [61]:
def upsample(df):

    major_class = df[df.y == 'no']
    minor_class = df[df.y == 'yes']
    
    df_minor_upsample = resample(minor_class, replace = True, n_samples = len(major_class), random_state = 42)
    df_upsample = pd.concat([major_class, df_minor_upsample])
    
    return df_upsample

In [62]:
# Upsampled
df_upsample_1 = df_full.copy(deep = True)

# Upsample, Transformations, Oulier Treatment and Logged  
df_upsample_4 = df_full.copy(deep = True)

# Upsample, Transformations, Oulier Treatment, Logged and Normalized
df_upsample_6 = df_full.copy(deep = True)

### 4.1.1 Upsample

In [63]:
df_upsample_1 = df_full.copy(deep = True)

In [64]:
df_upsample_1 = upsample(df_upsample_1)

In [65]:
df_upsample_1.y.value_counts()

no     36548
yes    36548
Name: y, dtype: int64

In [66]:
df_upsample_1 = pd.get_dummies(df_upsample_1, columns = categorical)

In [154]:
X = df_upsample_1.drop(['y'], axis = 1)
y = pd.get_dummies(df_upsample_1[['y']], drop_first = True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)

# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(X_train, y_train.values.ravel())

pred = rf.predict(X_test)

print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.968262653899
Precision --  0.999587118084
Recall --  0.940315898498
F1 Score --  0.969046030687
AUC --  434814.0


In [68]:
print(metrics.classification_report(pred, y_test))

             precision    recall  f1-score   support

          0       0.94      1.00      0.97      6896
          1       1.00      0.94      0.97      7724

avg / total       0.97      0.97      0.97     14620



** Random Search - Base Params **

In [156]:
# Train and Test data
X = df_upsample_1.drop(['y'], axis = 1)
y = pd.get_dummies(df_upsample_1[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting dataframe to numpy array
data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)
    
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.values.ravel());

# Predict test data
pred = rf.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.972708618331
Precision --  0.998073217726
Recall --  0.949587534372
F1 Score --  0.973226867074
AUC --  439811.0


** Random Search - Best Params **

In [157]:
# Train and Test data
X = df_upsample_1.drop(['y'], axis = 1)
y = pd.get_dummies(df_upsample_1[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting dataframe to numpy array
data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)
    
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(bootstrap = False, max_depth = 110,
 max_features = 0.2,
 min_samples_leaf = 5,
 min_samples_split = 5,
 n_estimators = 800, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.values.ravel());

# Predict test data
pred = rf.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.953283173735
Precision --  0.995458298927
Recall --  0.917544082202
F1 Score --  0.95491451581
AUC --  423795.0


** Grid Search - Best Params **

In [158]:
# Train and Test data
X = df_upsample_1.drop(['y'], axis = 1)
y = pd.get_dummies(df_upsample_1[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting dataframe to numpy array
data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)
    
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(bootstrap = False, max_depth = 110 ,
 max_features = 3,
 min_samples_leaf = 5,
 min_samples_split = 8,
 n_estimators = 100, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.values.ravel());

# Predict test data
pred = rf.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.939466484268
Precision --  0.979906413432
Recall --  0.905967680366
F1 Score --  0.941487603306
AUC --  418177.5


### 4.1.5 Upsample, Transformations, Outlier Treatment, Logged and Standardized

In [69]:
df_upsample_5 = df_full.copy(deep = True)

In [70]:
df_upsample_5 = upsample(df_upsample_5)

In [71]:
transformations(df_upsample_5)
outliers(df_upsample_5)
outliers_log(df_upsample_5)
standardization(df_upsample_5)

In [72]:
df_upsample_5.drop(['age_group', 'age_marital'], axis = 1, inplace = True)

In [73]:
df_upsample_5 = pd.get_dummies(df_upsample_5, columns = categorical)

In [74]:
X = df_upsample_5.drop(['y'], axis = 1)
y = pd.get_dummies(df_upsample_5[['y']], drop_first = True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)

# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(X_train, y_train.values.ravel())

pred = rf.predict(X_test)

print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

0.972503419973


In [75]:
print(metrics.classification_report(pred, y_test))

             precision    recall  f1-score   support

          0       0.95      1.00      0.97      6968
          1       1.00      0.95      0.97      7652

avg / total       0.97      0.97      0.97     14620



** Random Search - Base Params **

In [159]:
# Train and Test data
X = df_upsample_5.drop(['y'], axis = 1)
y = pd.get_dummies(df_upsample_5[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting dataframe to numpy array
data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)
    
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.values.ravel());

# Predict test data
pred = rf.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.971545827633
Precision --  0.998623726947
Recall --  0.947011224223
F1 Score --  0.972132904609
AUC --  436492.5


** Random Search - Best Params **

In [160]:
# Train and Test data
X = df_upsample_5.drop(['y'], axis = 1)
y = pd.get_dummies(df_upsample_5[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting dataframe to numpy array
data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)
    
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(bootstrap = False, max_depth = 110,
 max_features = 0.2,
 min_samples_leaf = 5,
 min_samples_split = 5,
 n_estimators = 800, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.values.ravel());

# Predict test data
pred = rf.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.950410396717
Precision --  0.977016240022
Recall --  0.927125506073
F1 Score --  0.951417275347
AUC --  429567.5


** Grid Search - Best Params **

In [161]:
# Train and Test data
X = df_upsample_5.drop(['y'], axis = 1)
y = pd.get_dummies(df_upsample_5[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting dataframe to numpy array
data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)
    
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(bootstrap = False, max_depth = 110 ,
 max_features = 3,
 min_samples_leaf = 5,
 min_samples_split = 8,
 n_estimators = 100, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.values.ravel());

# Predict test data
pred = rf.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.919357045144
Precision --  0.934902284613
Recall --  0.905854113882
F1 Score --  0.920149001016
AUC --  421887.5


### 4.1.6 Upsample, Transformations, Outlier Treatment, Logged and Normalized

In [76]:
df_upsample_6 = df_full.copy(deep = True)

In [77]:
df_upsample_6 = upsample(df_upsample_6)

In [78]:
transformations(df_upsample_6)
outliers(df_upsample_6)
outliers_log(df_upsample_6)
normalization(df_upsample_6)

In [79]:
df_upsample_6.drop(['age_group', 'age_marital'], axis = 1, inplace = True)

In [164]:
X = df_upsample_6.drop(['y'], axis = 1)
y = pd.get_dummies(df_upsample_6[['y']], drop_first = True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)

# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(X_train, y_train.values.ravel())

pred = rf.predict(X_test)

print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.97243502052
Precision --  0.998898981558
Recall --  0.948386253757
F1 Score --  0.972987465648
AUC --  437640.5


In [165]:
print(metrics.classification_report(pred, y_test))

             precision    recall  f1-score   support

          0       0.95      1.00      0.97      6967
          1       1.00      0.95      0.97      7653

avg / total       0.97      0.97      0.97     14620



** Random Search - Base Params **

In [166]:
# Train and Test data
X = df_upsample_6.drop(['y'], axis = 1)
y = pd.get_dummies(df_upsample_6[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting dataframe to numpy array
data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)
    
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.values.ravel());

# Predict test data
pred = rf.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.971614227086
Precision --  0.998623726947
Recall --  0.947134838794
F1 Score --  0.972198030415
AUC --  436747.5


** Random Search - Best Params **

In [167]:
# Train and Test data
X = df_upsample_6.drop(['y'], axis = 1)
y = pd.get_dummies(df_upsample_6[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting dataframe to numpy array
data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)
    
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(bootstrap = False, max_depth = 110,
 max_features = 0.2,
 min_samples_leaf = 5,
 min_samples_split = 5,
 n_estimators = 800, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.values.ravel());

# Predict test data
pred = rf.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.95047879617
Precision --  0.977016240022
Recall --  0.927246603971
F1 Score --  0.951481034714
AUC --  429695.0


** Grid Search - Best Params **

In [168]:
# Train and Test data
X = df_upsample_6.drop(['y'], axis = 1)
y = pd.get_dummies(df_upsample_6[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting dataframe to numpy array
data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)
    
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(bootstrap = False, max_depth = 110 ,
 max_features = 3,
 min_samples_leaf = 5,
 min_samples_split = 8,
 n_estimators = 100, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.values.ravel());

# Predict test data
pred = rf.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.919288645691
Precision --  0.934902284613
Recall --  0.905733333333
F1 Score --  0.920086685629
AUC --  421760.0


### 4.2 Downsample

In [83]:
def downsample(df):
    major_class = df[df.y == 'no']
    minor_class = df[df.y == 'yes']
    
    df_major_downsample = resample(major_class, replace = False, n_samples = len(minor_class), random_state = 42)
    df_downsample = pd.concat([df_major_downsample, minor_class])

    return df_downsample

In [84]:
# Downsample, Transformations
df_downsample_2 = df_full.copy(deep = True)

# Downsample, Transformations, Oulier Treatment, Logged and Standardized
df_downsample_5 = df_full.copy(deep = True)

# Downsample, Transformations, Oulier Treatment, Logged and Normalized
df_downsample_6 = df_full.copy(deep = True)

### 4.1.2 Downsample and Transformation

In [85]:
df_downsample_2 = df_full.copy(deep = True)

In [86]:
df_downsample_2 = downsample(df_downsample_2)
transformations(df_downsample_2)

In [87]:
df_downsample_2.drop(['age_group', 'age_marital'], axis = 1, inplace = True)

In [88]:
df_downsample_2 = pd.get_dummies(df_downsample_2, columns = categorical)

In [89]:
X = df_downsample_2.drop(['y'], axis = 1)
y = pd.get_dummies(df_downsample_2[['y']], drop_first = True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)

# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(X_train, y_train.values.ravel())

pred = rf.predict(X_test)

print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

0.883620689655


In [90]:
print(metrics.classification_report(pred, y_test))

             precision    recall  f1-score   support

          0       0.84      0.92      0.88       836
          1       0.93      0.86      0.89      1020

avg / total       0.89      0.88      0.88      1856



** Random Search - Base Params **

In [169]:
# Train and Test data
X = df_downsample_2.drop(['y'], axis = 1)
y = pd.get_dummies(df_downsample_2[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting dataframe to numpy array
data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)
    
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.values.ravel());

# Predict test data
pred = rf.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.853448275862
Precision --  0.86093418259
Recall --  0.851890756303
F1 Score --  0.856388595565
AUC --  52485.0


** Random Search - Best Params **

In [170]:
# Train and Test data
X = df_downsample_2.drop(['y'], axis = 1)
y = pd.get_dummies(df_downsample_2[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting dataframe to numpy array
data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)
    
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(bootstrap = False, max_depth = 110,
 max_features = 0.2,
 min_samples_leaf = 5,
 min_samples_split = 5,
 n_estimators = 800, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.values.ravel());

# Predict test data
pred = rf.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.889547413793
Precision --  0.943736730361
Recall --  0.853986551393
F1 Score --  0.896621280888
AUC --  52224.0


** Grid Search - Best Params **

In [171]:
# Train and Test data
X = df_downsample_2.drop(['y'], axis = 1)
y = pd.get_dummies(df_downsample_2[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting dataframe to numpy array
data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)
    
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(bootstrap = False, max_depth = 110 ,
 max_features = 3,
 min_samples_leaf = 5,
 min_samples_split = 8,
 n_estimators = 100, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.values.ravel());

# Predict test data
pred = rf.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.880387931034
Precision --  0.936305732484
Recall --  0.844827586207
F1 Score --  0.888217522659
AUC --  50692.5


### 4.1.5 Downsample, Transformations, Outlier Treatment, Logged and Standardized

In [91]:
df_downsample_5 = df_full.copy(deep = True)

In [92]:
df_downsample_5 = downsample(df_downsample_5)

In [93]:
# Function to perform standardization
def standardization_new(df):
    for n in numerical:
        col = df[[n]].values.astype(float)
        col_transformed = (preprocessing.StandardScaler()).fit_transform(col)
        # df[n+'_standardized'] = pd.DataFrame(col_transformed)
        df[n] = col_transformed

In [94]:
transformations(df_downsample_5)
outliers(df_downsample_5)
outliers_log(df_downsample_5)
standardization_new(df_downsample_5)

In [95]:
df_downsample_5.drop(['age_group', 'age_marital'], axis = 1, inplace = True)

In [96]:
df_downsample_5 = pd.get_dummies(df_downsample_5, columns = categorical)

In [97]:
X = df_downsample_5.drop(['y'], axis = 1)
y = pd.get_dummies(df_downsample_5[['y']], drop_first = True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)

# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.values.ravel())

pred = rf.predict(X_test)

print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

0.880387931034


In [98]:
print(metrics.classification_report(pred, y_test))

             precision    recall  f1-score   support

          0       0.84      0.91      0.87       836
          1       0.92      0.85      0.89      1020

avg / total       0.88      0.88      0.88      1856



** Random Search - Base Params **

In [172]:
# Train and Test data
X = df_downsample_5.drop(['y'], axis = 1)
y = pd.get_dummies(df_downsample_5[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting dataframe to numpy array
data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)
    
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.values.ravel());

# Predict test data
pred = rf.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.852370689655
Precision --  0.867303609342
Recall --  0.845755693582
F1 Score --  0.856394129979
AUC --  52605.5


** Random Search - Best Params **

In [173]:
# Train and Test data
X = df_downsample_5.drop(['y'], axis = 1)
y = pd.get_dummies(df_downsample_5[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting dataframe to numpy array
data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)
    
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(bootstrap = False, max_depth = 110,
 max_features = 0.2,
 min_samples_leaf = 5,
 min_samples_split = 5,
 n_estimators = 800, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.values.ravel());

# Predict test data
pred = rf.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.885775862069
Precision --  0.934182590234
Recall --  0.854368932039
F1 Score --  0.892494929006
AUC --  52738.0


** Grid Search - Best Params **

In [174]:
# Train and Test data
X = df_downsample_5.drop(['y'], axis = 1)
y = pd.get_dummies(df_downsample_5[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting dataframe to numpy array
data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)
    
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(bootstrap = False, max_depth = 110 ,
 max_features = 3,
 min_samples_leaf = 5,
 min_samples_split = 8,
 n_estimators = 100, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.values.ravel());

# Predict test data
pred = rf.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.875538793103
Precision --  0.918259023355
Recall --  0.848871442591
F1 Score --  0.882202957675
AUC --  51842.5


### 4.1.6 Downsample, Transformations, Outlier Treatment, Logged and Normalized

In [105]:
df_downsample_6 = df_full.copy(deep = True)

In [106]:
df_downsample_6 = downsample(df_downsample_6)

In [107]:
# Function to perform normalization
def normalization_new(df):
    for n in numerical:
        col = df[[n]].values.astype(float)
        col_transformed = (preprocessing.MinMaxScaler()).fit_transform(col)
        # df[n+'_normalized'] = pd.df(col_transformed)
        df[n] = col_transformed

In [108]:
transformations(df_downsample_6)
outliers(df_downsample_6)
outliers_log(df_downsample_6)

In [110]:
normalization_new(df_downsample_6)

In [111]:
df_downsample_6.drop(['age_group', 'age_marital'], axis = 1, inplace = True)

In [112]:
df_downsample_6 = pd.get_dummies(df_downsample_6, columns = categorical)

In [113]:
X = df_downsample_6.drop(['y'], axis = 1)
y = pd.get_dummies(df_downsample_6[['y']], drop_first = True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)

# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(X_train, y_train.values.ravel())

pred = rf.predict(X_test)

print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

0.880387931034


In [114]:
print(metrics.classification_report(pred, y_test))

             precision    recall  f1-score   support

          0       0.84      0.91      0.87       836
          1       0.92      0.85      0.89      1020

avg / total       0.88      0.88      0.88      1856



** Random Search - Base Params **

In [175]:
# Train and Test data
X = df_downsample_6.drop(['y'], axis = 1)
y = pd.get_dummies(df_downsample_6[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting dataframe to numpy array
data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)
    
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.values.ravel());

# Predict test data
pred = rf.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.852370689655
Precision --  0.867303609342
Recall --  0.845755693582
F1 Score --  0.856394129979
AUC --  52605.5


** Random Search - Best Params **

In [176]:
# Train and Test data
X = df_downsample_6.drop(['y'], axis = 1)
y = pd.get_dummies(df_downsample_6[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting dataframe to numpy array
data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)
    
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(bootstrap = False, max_depth = 110,
 max_features = 0.2,
 min_samples_leaf = 5,
 min_samples_split = 5,
 n_estimators = 800, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.values.ravel());

# Predict test data
pred = rf.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.885775862069
Precision --  0.934182590234
Recall --  0.854368932039
F1 Score --  0.892494929006
AUC --  52738.0


** Grid Search - Best Params **

In [177]:
# Train and Test data
X = df_downsample_6.drop(['y'], axis = 1)
y = pd.get_dummies(df_downsample_6[['y']], drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Converting dataframe to numpy array
data = [X_train, X_test, y_train, y_test]
for d in data:
    d = np.array(d)
    
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(bootstrap = False, max_depth = 110 ,
 max_features = 3,
 min_samples_leaf = 5,
 min_samples_split = 8,
 n_estimators = 100, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.values.ravel());

# Predict test data
pred = rf.predict(X_test)

# Accuray Score
print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
print("Precision -- ", metrics.precision_score(pred, y_test))
print("Recall -- ", metrics.recall_score(pred, y_test))
print("F1 Score -- ", metrics.f1_score(pred, y_test))
print("AUC -- ", metrics.auc(pred, y_test))

Accuracy --  0.875538793103
Precision --  0.918259023355
Recall --  0.848871442591
F1 Score --  0.882202957675
AUC --  51842.5


After builds models with various transformations, feature engineering techniques and their combinations, here are that were selected based on recall, precision and f1 scores.

<img src="img/models and scores.png">

** Note: ** <br>
Primary Models - Highlighted in yellow <br>
Experimenting Models - Highlighted in blue

## 5. Applying Best Params from Random Search and Grid Search using a for-loop

In [187]:
all_dfs = [df_full, df_age_1, df_age_2, df_upsample_1, df_upsample_5, df_upsample_6, df_downsample_2, df_downsample_5, df_downsample_6]

In [188]:
# Random Search
for df in all_dfs:
    # Train and Test data
    X = df.drop(['y'], axis = 1)
    y = pd.get_dummies(df[['y']], drop_first = True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

    # Converting dataframe to numpy array
    data = [X_train, X_test, y_train, y_test]
    for d in data:
        d = np.array(d)

    # Instantiate model with 1000 decision trees
    rf = RandomForestClassifier(random_state = 42)

    # Train the model on training data
    rf.fit(X_train, y_train.values.ravel());

    # Predict test data
    pred = rf.predict(X_test)

    # Accuray Score
    print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
    print("Precision -- ", metrics.precision_score(pred, y_test))
    print("Recall -- ", metrics.recall_score(pred, y_test))
    print("F1 Score -- ", metrics.f1_score(pred, y_test))
    print("AUC -- ", metrics.auc(pred, y_test))
    print("----------------------------------------------------------------------------------")

Accuracy --  0.908351541636
Precision --  0.404661016949
Recall --  0.664347826087
F1 Score --  0.502962475313
AUC --  46974.0
----------------------------------------------------------------------------------
Accuracy --  0.907259043457
Precision --  0.407838983051
Recall --  0.652542372881
F1 Score --  0.501955671447
AUC --  47360.0
----------------------------------------------------------------------------------
Accuracy --  0.908108764263
Precision --  0.423728813559
Recall --  0.652528548124
F1 Score --  0.513808606294
AUC --  48131.5
----------------------------------------------------------------------------------
Accuracy --  0.972708618331
Precision --  0.998073217726
Recall --  0.949587534372
F1 Score --  0.973226867074
AUC --  439811.0
----------------------------------------------------------------------------------
Accuracy --  0.971545827633
Precision --  0.998623726947
Recall --  0.947011224223
F1 Score --  0.972132904609
AUC --  436492.5
-------------------------------

In [186]:
df_downsample_6.head()

,age,duration,dcontacts,pdays,pcontacts,evr,cpi,cci,euribor,employees,y,job_jl1,job_jl2,job_jl3,job_jl4,marital_divorced,marital_married,marital_single,marital_unknown,education_el1,education_el2,education_el3,education_el4,default_no,default_unknown,housing_no,housing_unknown,housing_yes,personal_no,personal_unknown,personal_yes,contact_type_cellular,contact_type_telephone,month_ml1,month_ml2,month_ml3,day_weekday_1,day_weekday_2,poutcome_failure,poutcome_nonexistent,poutcome_success
4532,0.568246,0.375000,0.000000,1.0,0.0,1.0,0.996697,0.623529,1.0,0.836562,no,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,1,0,0,1,1,0,0,1,0
7208,0.264092,0.019231,0.000000,1.0,0.0,1.0,0.954837,0.252941,1.0,1.000000,no,0,1,0,0,0,1,0,0,0,0,0,1,0,1,1,0,0,1,0,0,1,0,0,0,1,1,0,0,1,0
24121,0.659779,0.096154,0.000000,1.0,0.0,1.0,0.692998,0.641176,1.0,1.000000,no,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,1,0,0,1,0,0,0,1,1,0,0,1,0
26957,0.378646,0.096154,0.386853,1.0,0.0,1.0,1.000000,0.305882,1.0,1.000000,no,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,1,0
27762,0.412235,0.278846,0.386853,1.0,0.0,1.0,0.996697,0.623529,1.0,0.836562,no,0,0,0,1,0,1,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,1,0,0,1,1,0,0,1,0


In [189]:
# Random Search - Best Params
for df in all_dfs:
    # Train and Test data
    X = df.drop(['y'], axis = 1)
    y = pd.get_dummies(df[['y']], drop_first = True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

    # Converting dataframe to numpy array
    data = [X_train, X_test, y_train, y_test]
    for d in data:
        d = np.array(d)

    # Instantiate model with 1000 decision trees
    rf = RandomForestClassifier(bootstrap = False, max_depth = 110,
     max_features = 0.2,
     min_samples_leaf = 5,
     min_samples_split = 5,
     n_estimators = 800, random_state = 42)

    # Train the model on training data
    rf.fit(X_train, y_train.values.ravel());

    # Predict test data
    pred = rf.predict(X_test)

    # Accuray Score
    print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
    print("Precision -- ", metrics.precision_score(pred, y_test))
    print("Recall -- ", metrics.recall_score(pred, y_test))
    print("F1 Score -- ", metrics.f1_score(pred, y_test))
    print("AUC -- ", metrics.auc(pred, y_test))
    print("----------------------------------------------------------------------------------")

Accuracy --  0.916484583637
Precision --  0.503177966102
Recall --  0.684438040346
F1 Score --  0.579975579976
AUC --  56956.5
----------------------------------------------------------------------------------
Accuracy --  0.917212915756
Precision --  0.512711864407
Recall --  0.685552407932
F1 Score --  0.586666666667
AUC --  58362.5
----------------------------------------------------------------------------------
Accuracy --  0.91709152707
Precision --  0.507415254237
Recall --  0.687230989957
F1 Score --  0.583790371725
AUC --  57850.0
----------------------------------------------------------------------------------
Accuracy --  0.953283173735
Precision --  0.995458298927
Recall --  0.917544082202
F1 Score --  0.95491451581
AUC --  423795.0
----------------------------------------------------------------------------------
Accuracy --  0.950410396717
Precision --  0.977016240022
Recall --  0.927125506073
F1 Score --  0.951417275347
AUC --  429567.5
---------------------------------

In [ ]:
df = pd.get_dummies(df, columns = categorical)

In [191]:
# Grid Search - Best Params
for df in all_dfs:
    
    # Train and Test data
    X = df.drop(['y'], axis = 1)
    y = pd.get_dummies(df[['y']], drop_first = True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

    # Converting dataframe to numpy array
    data = [X_train, X_test, y_train, y_test]
    for d in data:
        d = np.array(d)

    # Instantiate model with 1000 decision trees
    rf = RandomForestClassifier(bootstrap = False, max_depth = 110 ,
     max_features = 3,
     min_samples_leaf = 5,
     min_samples_split = 8,
     n_estimators = 100, random_state = 42)

    # Train the model on training data
    rf.fit(X_train, y_train.values.ravel());

    # Predict test data
    pred = rf.predict(X_test)

    # Accuray Score
    print("Accuracy -- ", metrics.accuracy_score(pred, y_test))
    print("Precision -- ", metrics.precision_score(pred, y_test))
    print("Recall -- ", metrics.recall_score(pred, y_test))
    print("F1 Score -- ", metrics.f1_score(pred, y_test))
    print("AUC -- ", metrics.auc(pred, y_test))
    print("----------------------------------------------------------------------------------")

Accuracy --  0.906652100024
Precision --  0.273305084746
Recall --  0.756598240469
F1 Score --  0.401556420233
AUC --  32250.5
----------------------------------------------------------------------------------
Accuracy --  0.909079873756
Precision --  0.323093220339
Recall --  0.734939759036
F1 Score --  0.448859455482
AUC --  37883.0
----------------------------------------------------------------------------------
Accuracy --  0.908837096383
Precision --  0.301906779661
Recall --  0.755968169761
F1 Score --  0.431491294474
AUC --  35069.0
----------------------------------------------------------------------------------
Accuracy --  0.939466484268
Precision --  0.979906413432
Recall --  0.905967680366
F1 Score --  0.941487603306
AUC --  418177.5
----------------------------------------------------------------------------------
Accuracy --  0.919357045144
Precision --  0.934902284613
Recall --  0.905854113882
F1 Score --  0.920149001016
AUC --  421887.5
-------------------------------